In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic').getOrCreate()

24/11/23 00:37:03 WARN Utils: Your hostname, Franciss-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.40 instead (on interface en0)
24/11/23 00:37:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/11/23 00:37:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/23 00:37:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/23 00:37:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/23 00:37:05 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/11/23 00:37:05 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24

In [2]:
from pyspark.sql.functions import col,current_timestamp,count,countDistinct,sum,aggregate,desc,rank,when
from pyspark.sql.window import Window

In [2]:
race_result_agg_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../tbls/presentation/race_results_twoyears/race_results_twoyears.csv')\
    .filter('race_year=2020')

In [5]:
driver_standing_df = race_result_agg_df.groupBy('race_year','team')\
    .agg(sum('points').alias('total_points'),\
    count(when(col('position') ==1,True)).alias('wins'))

In [6]:
window_spec = Window.partitionBy('race_year')\
    .orderBy(desc('total_points'),desc('wins'))
driver_standing_df.withColumn('rank',rank().over(window_spec)).show(50)

+---------+------------+------------+----+----+
|race_year|        team|total_points|wins|rank|
+---------+------------+------------+----+----+
|     2020|    Mercedes|       573.0|  13|   1|
|     2020|    Red Bull|       319.0|   2|   2|
|     2020|Racing Point|       210.0|   1|   3|
|     2020|     McLaren|       202.0|   0|   4|
|     2020|     Renault|       181.0|   0|   5|
|     2020|     Ferrari|       131.0|   0|   6|
|     2020|  AlphaTauri|       107.0|   1|   7|
|     2020|  Alfa Romeo|         8.0|   0|   8|
|     2020|Haas F1 Team|         3.0|   0|   9|
|     2020|    Williams|         0.0|   0|  10|
+---------+------------+------------+----+----+

